In [1]:
cd .. 

/Users/johanbarreiro/Documents/GitHub/vl_optimizer


# Evaluating Results

## Create ground Truth CSV

In [2]:
import pandas as pd
import os

df = pd.read_csv('data/processed_data/industrial_sites_processed/2024-07-07T16-56-34_industrial_site2_processed.csv')



In [3]:
def join_csv_on_time_column(directory='.'):
    # Get all CSV files in the directory
    csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    
    # Initialize an empty dataframe
    merged_df = pd.DataFrame()
    
    # Iterate through each CSV file and merge them on the 'Time' column
    for csv_file in csv_files:
        df = pd.read_csv(os.path.join(directory, csv_file))  
        
        # Check if 'Time' column exists in the dataframe
        if 'Time' not in df.columns:
            print(f"'Time' column not found in {csv_file}")
            continue
        
        # Merge the dataframes
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on='Time', how='outer')
    
    return merged_df

# Run the function and store the result
merged_dataframe = join_csv_on_time_column(directory='data/synthesized_data')



In [4]:
# Choose the 24-hour period you want to optimize (for example, starting on 2024-07-01 00:00:00)
start_date = '2023-06-01 00:00:00'
end_date = '2023-07-01 00:00:00'

# Filter the data for the chosen 24-hour period
ground_truth = df[(df['Time'] >= start_date) & (df['Time'] < end_date)]

ground_truth = pd.merge(ground_truth, merged_dataframe, on='Time', how='left')
ground_truth['heat_index'] = ground_truth['heat_index_x']
ground_truth['03 Chiller Group_Electric_Active Energy (kWh)'] = ground_truth['03 Chiller Group_Electric_Active Energy (kWh)_x']
ground_truth = ground_truth.drop(columns=['heat_index_x', 'heat_index_y', '03 Chiller Group_Electric_Active Energy (kWh)_x', '03 Chiller Group_Electric_Active Energy (kWh)_y', 'total_consumption','target_consumption'])

# Ensure that the data is sorted by timestamp and reset the index
ground_truth = ground_truth.sort_values(by='Time').reset_index(drop=True)




ground_truth.to_csv('optimizer/ground_truth/ground_truth_hourly.csv', index = False)






In [5]:
ground_truth = ground_truth[['Time', 'energy_cost']]

# Aggregate the data to daily
ground_truth['Time'] = pd.to_datetime(ground_truth['Time'])
ground_truth = ground_truth.set_index('Time')
ground_truth = ground_truth.resample('D').sum()

ground_truth.to_csv('optimizer/ground_truth/ground_truth_daily.csv')